<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/submissions/Turkish_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focal-loss
!pip install keras-tcn==2.8.3

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D
from keras.optimizers import RMSprop,Adam,Adadelta
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import csv
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from keras import backend as K
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


# Load data

In [4]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [5]:
ls

cc.tr.300.vec              offenseval-tr-testset-v1.tsv   tcn.npy
cc.tr.300.vec.gz           offenseval-tr-training-v1.tsv
offenseval-annotation.txt  readme-trainingset-tr.txt


In [6]:
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
train.drop(train.tail(n=1).index,inplace=True)
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)
print(len(train))
train.head()

31755


,id,tweet,subtask_a
27201,11608,anasayfama erkek terimleri düştü şimdi kusucam...,NOT
19899,15359,der. biz bu zihniyeti çok iyi tanıyoruz. km ...,OFF
25974,32255,kaçırıyorsun uykularımı bu kadar çok görünme c...,NOT
754,20552,allah aşkına burada soru cevap yapmak niye ya ...,OFF
29368,20417,işınlanmayla anca gelirim yoksa yolda erime i...,NOT


In [7]:
test=pd.read_csv( 'offenseval-tr-testset-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()
print(test.head())
ids=test['id']
test=test['tweet']


      id                                              tweet
0  41993          sayın başkanım bu şekilde devam inşallah👏
1  23000  herkes gevşekliği kadar duyar kasıyor,hayat bö...
2  42478  olgun ilişkisi olan arkadaş size en güzel hedi...
3  21748    burada atıp tutacağına o kötü koşullarda  ku...
4  13607        i̇şte o onur dediğin sende yok sorun o işte


#Label Encoding

In [0]:
labels=train['subtask_a']
train=train['tweet']

In [9]:
le=LabelEncoder()
labels=le.fit_transform(labels)
print(len(labels))

31755


# Common Parameters

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

## Cylic learning rate

In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
from keras.callbacks import Callback
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.005,
                        step_size=4., mode='exp_range',
                        gamma=0.99994)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [14]:
len(sequences_matrix),len(labels)

(31755, 31755)

In [15]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
test_sequences = tokenize.texts_to_sequences(test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

In [17]:
len(X_test_sequences)

3528

# Embeddings

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [19]:
embeddings=build_matrix('cc.tr.300.vec', word_index)
embeddings.shape

(10001, 300)

# Base Models

In [20]:
train.shape,labels.shape

((31755,), (31755,))

## Count Vectorizer

In [0]:

def logistic_param_selection(X, y, func,nfolds):
    C= [7,8,9,10,12,15,20,25]
    param_grid = {'C': C}
    grid_search = GridSearchCV(make_pipeline(func, LogisticRegression(solver='lbfgs',max_iter=500,class_weight=class_weights)),
                    param_grid={'logisticregression__C':[10,12,15,20,25]}, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    return grid_search.best_score_,grid_search.best_params_



In [22]:
cv_score,c=logistic_param_selection(train,labels,CountVectorizer(),5)
cv_score

0.7118123864066545

In [0]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train)
X_train_cv = count_vectorizer.transform(train)


In [0]:
X_test_cv=  count_vectorizer.transform(test)

In [25]:
cv_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500,class_weight=class_weights)
cv_classifier.fit(X_train_cv, labels)

LogisticRegression(C=10,
                   class_weight={0: 0.6196339369341243, 1: 2.5897080411025932},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=500, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [0]:
cv_test=cv_classifier.predict_proba(X_test_cv)[:,1]

## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(train)
train_word_features = word_vectorizer.transform(train)


In [0]:
test_word_features = word_vectorizer.transform(test)

In [29]:
tfw_score,c=logistic_param_selection(train,labels,word_vectorizer,5)
tfw_score

0.687353952843696

In [30]:
tfw_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfw_classifier.fit(train_word_features, labels)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
tfw_test = tfw_classifier.predict_proba(test_word_features)[:,1]

## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(train)
train_char_features = char_vectorizer.transform(train)


In [0]:
test_char_features = char_vectorizer.transform(test)

In [34]:
tfc_score,c=logistic_param_selection(train,labels,char_vectorizer,5)
tfc_score

0.7319249634827353

In [35]:
tfc_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfc_classifier.fit(train_char_features, labels)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
tfc_test = tfc_classifier.predict_proba(test_char_features)[:,1]

# Models

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(CuDNNGRU(50, return_sequences=True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic
from keras.layers import Layer,Lambda
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
    
    return model

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


# Train model

##GRU

In [42]:
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=2020)
gru_scores = []
gru_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  gru_test.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 25404 samples, validate on 6351 samples
Epoch 1/10
 - 22s - loss: 0.1034 - acc: 0.8436 - val_loss: 0.0965 - val_acc: 0.8523
Epoch 2/10
 - 15s - loss: 0.0946 - acc: 0.8581 - val_loss: 0.0948 - val_acc: 0.8581
Epoch 3/10
 - 15s - loss: 0.0910 - acc: 0.8618 - val_loss: 0.0939 - val_acc: 0.8551
Epoch 4/10
 - 15s - loss: 0.0889 - acc: 0.8660 - val_loss: 0.0934 - val_acc: 0.8586
Epoch 5/10
 - 15s - loss: 0.0847 - acc: 0.8718 - val_loss: 0.0954 - val_acc: 0.8528
Epoch 6/10
 - 15s - loss: 0.0809 - acc: 0.8776 - val_loss: 0.0994 - val_acc: 0.8567
Epoch 7/10
 - 15s - loss: 0.0765 - acc: 0.8851 - val_loss: 0.1015 - val_acc: 0.8566
Epoch 8/10
 - 15s - loss: 0.0696 - acc: 0.8959 - val_loss: 0.1059 - val_acc: 0.8449
Epoch 9/10
 - 15s - loss: 0.0634 - acc: 0.9049 - val_loss: 0.1170 - val_acc: 0.8466
Epoch 10/10
 - 15s - loss: 0.0547 - acc: 0.9179 - val_loss: 0.1262 - val_acc: 0.8372
3528/3528 [==============================] - 1s 248us/step
Train on 25404 samples, validate on 6351 samples
Ep

In [43]:
len(gru_test),gru_test[0].shape,gru_scores

(5,
 (3528,),
 [0.7120300302154442,
  0.7268300905413215,
  0.7244354173875709,
  0.7251543157752521,
  0.7209336105107337])

## GRU attention

In [44]:
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=2020)
atten_scores = []
atten_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  attn_gru_model=model_gru_attn()
  
  attn_gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = attn_gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  atten_test.append(attn_gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  atten_scores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 25404 samples, validate on 6351 samples
Epoch 1/10
 - 16s - loss: 0.1061 - acc: 0.8386 - val_loss: 0.1018 - val_acc: 0.8463
Epoch 2/10
 - 15s - loss: 0.0957 - acc: 0.8553 - val_loss: 0.0974 - val_acc: 0.8507
Epoch 3/10
 - 15s - loss: 0.0919 - acc: 0.8620 - val_loss: 0.0947 - val_acc: 0.8572
Epoch 4/10
 - 15s - loss: 0.0900 - acc: 0.8642 - val_loss: 0.0944 - val_acc: 0.8570
Epoch 5/10
 - 15s - loss: 0.0872 - acc: 0.8671 - val_loss: 0.0938 - val_acc: 0.8578
Epoch 6/10
 - 15s - loss: 0.0851 - acc: 0.8726 - val_loss: 0.0957 - val_acc: 0.8575
Epoch 7/10
 - 16s - loss: 0.0810 - acc: 0.8778 - val_loss: 0.0965 - val_acc: 0.8553
Epoch 8/10
 - 16s - loss: 0.0773 - acc: 0.8830 - val_loss: 0.1003 - val_acc: 0.8501
Epoch 9/10
 - 16s - loss: 0.0722 - acc: 0.8916 - val_loss: 0.1099 - val_acc: 0.8537
Epoch 10/10
 - 16s - loss: 0.0671 - acc: 0.8977 - val_loss: 0.1112 - val_acc: 0.8477
3528/3528 [==============================] - 1s 279us/step
Train on 25404 samples, validate on 6351 samples
Ep

In [45]:
atten_scores 


[0.7271683853832069,
 0.7126559597688719,
 0.7289190312081708,
 0.6911069634042271,
 0.7321550759235212]

##TCN

In [46]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
tcn_scores = []
tcn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_tcn(embeddings)
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  tcn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  tcn_scores.append(f1)
  keras.backend.clear_session()

Train on 25404 samples, validate on 6351 samples
Epoch 1/10
 - 17s - loss: 0.1094 - acc: 0.8344 - val_loss: 0.0999 - val_acc: 0.8493
Epoch 2/10
 - 11s - loss: 0.0982 - acc: 0.8551 - val_loss: 0.0994 - val_acc: 0.8465
Epoch 3/10
 - 11s - loss: 0.0951 - acc: 0.8574 - val_loss: 0.1007 - val_acc: 0.8551
Epoch 4/10
 - 11s - loss: 0.0931 - acc: 0.8596 - val_loss: 0.0962 - val_acc: 0.8548
Epoch 5/10
 - 11s - loss: 0.0919 - acc: 0.8623 - val_loss: 0.0964 - val_acc: 0.8573
Epoch 6/10
 - 11s - loss: 0.0899 - acc: 0.8662 - val_loss: 0.0996 - val_acc: 0.8547
Epoch 7/10
 - 11s - loss: 0.0878 - acc: 0.8677 - val_loss: 0.0965 - val_acc: 0.8555
Epoch 8/10
 - 11s - loss: 0.0852 - acc: 0.8737 - val_loss: 0.0991 - val_acc: 0.8503
Epoch 9/10
 - 11s - loss: 0.0832 - acc: 0.8745 - val_loss: 0.0967 - val_acc: 0.8523
Epoch 10/10
 - 11s - loss: 0.0804 - acc: 0.8788 - val_loss: 0.1006 - val_acc: 0.8499
3528/3528 [==============================] - 1s 214us/step
Train on 25404 samples, validate on 6351 samples
Ep

##KIM

In [47]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
kim_scores = []
kim_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_kim()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  kim_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  kim_scores.append(f1)
  keras.backend.clear_session()

Train on 25404 samples, validate on 6351 samples
Epoch 1/10
 - 12s - loss: 0.1509 - acc: 0.7696 - val_loss: 0.1095 - val_acc: 0.8333
Epoch 2/10
 - 10s - loss: 0.1067 - acc: 0.8372 - val_loss: 0.1017 - val_acc: 0.8411
Epoch 3/10
 - 10s - loss: 0.0965 - acc: 0.8506 - val_loss: 0.1000 - val_acc: 0.8471
Epoch 4/10
 - 10s - loss: 0.0891 - acc: 0.8630 - val_loss: 0.1051 - val_acc: 0.8455
Epoch 5/10
 - 10s - loss: 0.0829 - acc: 0.8719 - val_loss: 0.0989 - val_acc: 0.8485
Epoch 6/10
 - 10s - loss: 0.0762 - acc: 0.8810 - val_loss: 0.1007 - val_acc: 0.8430
Epoch 7/10
 - 10s - loss: 0.0705 - acc: 0.8932 - val_loss: 0.1075 - val_acc: 0.8490
Epoch 8/10
 - 10s - loss: 0.0637 - acc: 0.9029 - val_loss: 0.1163 - val_acc: 0.8492
Epoch 9/10
 - 10s - loss: 0.0577 - acc: 0.9134 - val_loss: 0.1106 - val_acc: 0.8449
Epoch 10/10
 - 10s - loss: 0.0531 - acc: 0.9200 - val_loss: 0.1178 - val_acc: 0.8252
3528/3528 [==============================] - 0s 117us/step
Train on 25404 samples, validate on 6351 samples
Ep

#**Result**

# Result

In [0]:
gru_scores=np.average(np.array(gru_scores))
atten_scores=np.average(np.array(atten_scores))
tcn_scores=np.average(np.array(tcn_scores))
kim_scores=np.average(np.array(kim_scores))

In [49]:
print('f1 score of count vec' ,cv_score)
print('f1 score of word tfidf' ,tfw_score)
print('f1 score of char tfidf' ,tfc_score)

print('f1 score of RNN' ,gru_scores)
print('f1 score of gated attention',atten_scores )
print('f1 score of tcn',tcn_scores)
print('f1 score of kim',kim_scores)


f1 score of count vec 0.7118123864066545
f1 score of word tfidf 0.687353952843696
f1 score of char tfidf 0.7319249634827353
f1 score of RNN 0.7218766928860645
f1 score of gated attention 0.7184010831375996
f1 score of tcn 0.7195472185997964
f1 score of kim 0.709329186768878


In [0]:
kim_test_avg=np.mean(np.array(kim_test),axis=0)
gru_test_avg=np.mean(np.array(gru_test),axis=0)
atten_test_avg=np.mean(np.array(atten_test),axis=0)
tcn_test_avg=np.mean(np.array(tcn_test),axis=0)


In [51]:
print(cv_test[5:15].round())
print(tfw_test[5:15].round())
print(tfc_test[5:15].round())
print(gru_test_avg[5:15].round())
print(atten_test_avg[5:15].round())
print(kim_test_avg[5:15].round())
print(tcn_test_avg[5:15].round())

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [52]:
wt=gru_scores+atten_scores+cv_score+tfw_score+tfc_score+tcn_scores+kim_scores
wt

5.000245484125424

In [0]:
y_test=gru_scores/wt * gru_test_avg + atten_scores/wt * atten_test_avg + cv_score/wt * cv_test + tfw_score/wt *tfw_test + tfc_score/wt *tfc_test +kim_scores/wt * kim_test_avg + tcn_scores/wt *tcn_test_avg

In [54]:
y_test[5:15].round()

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
def decode(y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [56]:
y_test=decode(y_test)
y_test.head()



,id,label
0,41993,NOT
1,23000,NOT
2,42478,NOT
3,21748,NOT
4,13607,NOT


In [57]:
y_test[5:15]

,id,label
5,28439,NOT
6,14664,OFF
7,22741,NOT
8,40997,NOT
9,15724,NOT
10,26615,NOT
11,36970,NOT
12,40935,NOT
13,23610,NOT
14,22870,NOT


# Submit file

In [0]:
y_test.to_csv('/content/final.csv',index=False,header=None)


# Reset

In [0]:
# import keras
# keras.backend.clear_session()